#  The cost of data accesses --- caches and prefetches

# 1. Measuring latency
* let's measure the latency of data access by a load instruction, which depends on _where_ the data comes from (L1 cache, L2 cache, L3 cache, and the main memory)
* we do so by _pointer chasing_, which is essentially the following loop
```
for many times {
  p = p->next;
}
* we expect this code to take (the latency of the load instruction for p->next) $\times$ the number of iterations
```
* for a reason not very relevant here, the following program actually does it by 
```
repeat {
  k = a[k]
}
```
(the difference is not important here. both end up doing almost the same thing)
* the point here is data access (`a[k]`) needs the previous access to have been finished, as it uses `k` _to determine the address accessed_ and `k` is generated by the previous access


In [ ]:
%matplotlib widget

# trick to make the visualization nicer

In [ ]:
%%writefile ptr_chase.cc
long scan(long * a, long n) {
  long k = 0;
  for (long i = 0; i < n; i++) {
    k = a[k];
  }
  return k;
}


* the main function including command line argument processing, setting up the array, and measuring cycles and other things 
* by default, it measures 
  * the number of instructions executed (instructions)
  * the number of core cycles (cycles)
  * L1 cache misses (L1-dcache-load-misses)
  * what is apparently L3 cache misses (cache-misses, with a caveat)

* in this environment, I don't know how to measure L2 cache misses
* `perf list` command shows everything we can measure, but there are no event names that specifically correspond to L2 cache misses

In [ ]:
%%writefile mem_main_cpu.cc
#include <assert.h>
#include <getopt.h>
#include <stdio.h>
#include <stdlib.h>
#include "event.h"

#if __AVX512F__
enum { vwidth = 64 };
enum { L = vwidth / sizeof(long) };
typedef long longv __attribute__((vector_size(vwidth)));
#else
#error "__AVX512F__ not defined. forgot to pass -mavx512f?"
#endif

enum { line_size = 64 };

/* make a list of n cells, each of which points
   to s cells after */
long * mk_list(long n, long s) {
  long * a = (long *)aligned_alloc(vwidth, sizeof(long) * n);
  for (long i = 0; i < n; i++) {
    a[i] = -1;
  }
  long k = 0;
  for (long i = 0; i < n; i++) {
    if (a[k] != -1) {
      assert(k == 0);
      break;
    }
    long k_next = (k + s) % n;
    a[k] = k_next;
    k = k_next;
  }
  return a;
}

long count_distinct_lines(long * a, long n) {
  long d = 0;
  long elems_per_line = line_size / sizeof(long);
  for (long i = 0; i < n; i += elems_per_line) {
    for (long j = 0; j < elems_per_line; j++) {
      if (a[i + j] != -1) {
        d++;
        break;
      }
    }
  }
  return d;
}

longv& V(long& p) {
  return *((longv *)&p);
}

long scan(long * a, long n);

struct opts {
  long size;
  long stride;
  long n_accesses;
  const char * events;
  opts() {
    size       = (1 << 15);
    stride     = line_size;
    n_accesses = 100 * 1000 * 1000;
    //events     = strdup("instructions,cycles,L1-dcache-load-misses,l2_lines_in.all,offcore_requests.l3_miss_demand_data_rd");
    events     = strdup("instructions,cycles,L1-dcache-load-misses,cache-misses");
  }
  ~opts() {
    free((void *)events);
  }
};

void usage(char * prog) {
  opts o;
  fprintf(stderr, "usage:\n");
  fprintf(stderr, "  %s [options]\n", prog);
  fprintf(stderr, "options:\n");
  fprintf(stderr, "  -n,--size N (%ld)\n", o.size);
  fprintf(stderr, "  -s,--stride N (%ld)\n", o.stride);
  fprintf(stderr, "  -a,--n-accesses N (%ld)\n", o.n_accesses);
  fprintf(stderr, "  -e,--events ev,ev,ev,.. (%s)\n", o.events);
}

long round_up(long x, long a) {
  x += a - 1;
  x -= x % a;
  return x;
}

opts * parse_cmdline(int argc, char * const * argv, opts * o) {
  static struct option long_options[] = {
    {"size",       required_argument, 0, 'n' },
    {"stride",     required_argument, 0, 's' },
    {"n_accesses", required_argument, 0, 'a' },
    {"events",     required_argument, 0, 'e' },
    {0,            0,                 0,  0 }
  };

  while (1) {
    int option_index = 0;
    int c = getopt_long(argc, argv, "n:s:a:e:",
			long_options, &option_index);
    if (c == -1) break;

    switch (c) {
    case 'n':
      o->size = atol(optarg);
      break;
    case 's':
      o->stride = atol(optarg);
      break;
    case 'a':
      o->n_accesses = atol(optarg);
      break;
    case 'e':
      free((void *)o->events);
      o->events = strdup(optarg);
      break;
    default:
      usage(argv[0]);
      exit(1);
    }
  }
  o->size = round_up(o->size, sizeof(long));
  o->stride = round_up(o->stride, sizeof(long));
  return o;
}

int main(int argc, char ** argv) {
  opts o;
  parse_cmdline(argc, argv, &o);
  long n_elements = o.size / sizeof(long);
  long stride = o.stride / sizeof(long);
  long * a = mk_list(n_elements, stride);
  long d = count_distinct_lines(a, n_elements);
  printf("size            : %ld\n", o.size);
  printf("stride          : %ld\n", o.stride);
  printf("distinct_blocks : %ld\n", d);
  printf("accesses        : %ld\n", o.n_accesses);
  fflush(stdout);
  perf_event_counters_t pc = mk_perf_event_counters(o.events);
  perf_event_values_t v0 = perf_event_counters_get(pc);
  long l = scan(a, o.n_accesses);
  perf_event_values_t v1 = perf_event_counters_get(pc);
  printf("last element   : %ld\n", l);
  for (int i = 0; i < pc.n; i++) {
    printf("%s : %lld\n", pc.events[i], v1.values[i] - v0.values[i]);
  }
  perf_event_counters_destroy(pc);
  assert(l == (o.n_accesses * stride % n_elements));
  printf("OK\n");
  free(a);
  return 0;
}


* compile (either by g++, clang++, or nvc++, whichever you like)

* note: we are not using bash kernel in SoS notebook but python kernel, for the purpose of better visualization of the graph
  * `!` is the trick to execute a shell command within a python kernel


In [ ]:
! g++                          -Wall -O3 -mavx512f mem_main_cpu.cc ptr_chase.cc -o ptr_chase -lpfm
! # /home/share/llvm/bin/clang++ -Wall -O3 -mavx512f mem_main_cpu.cc ptr_chase.cc -o ptr_chase -lpfm
! # /opt/nvidia/hpc_sdk/Linux_x86_64/22.11/compilers/bin/nvc++ -Wall -O3 -mavx512f mem_main_cpu.cc ptr_chase.cc -o ptr_chase -lpfm


* run for help


In [ ]:
! ./ptr_chase --help


```
./ptr_chase -n N -s S -a A
```
creates an array of $N$ bytes and accesses it with $S$-byte stride $A$ times.

* the command below creates an array of 16KB ($<$ L1 cache size) accesses elements 100M times with 64-byte stride ($=$ cache line size)


In [ ]:
! ./ptr_chase -n $((16 * 1024)) -s 64 -a $((100 * 1000 * 1000))


* observe
   * the number of instructions
   * the number of cycles (core cycles)
and calculate the latency of a single data access (`a[k]`)

* witness the assembly code, generated by g++ and/or clang++ to make sense of the number of instructions and the latency


In [ ]:
! g++                          -Wall -O3 -mavx512f ptr_chase.cc -S
! # /home/share/llvm/bin/clang++ -Wall -O3 -mavx512f ptr_chase.cc -S
! # /opt/nvidia/hpc_sdk/Linux_x86_64/22.11/compilers/bin/nvc++ -Wall -O3 -mavx512f ptr_chase.cc -S


##  A few notes on how to make that happen
* it creates an array of `long` having size of $N$ bytes ($N$ is rounded up to the next multiple of `sizeof(long)`; e.g., $N=12345$ -&gt; an array of 12352 bytes = 1544 `long` elements will be created)
* initialize the array (let's call it$a$) so that elements are accessed with $S$-byte stride. Again, $S$ is rounded up to the next multiple of `sizeof(long)`; e.g., $S=123$ -&gt; elements are accessed with 128-byte stride, which is 16-element stride.  That is, $a[k] = k + 16$.  To accommodate the case where $(k + 16)$ overflows the array, it is actually initialized with $a[k] = (k + 16) \mbox{ (mod the number of elements of the array)}$.
* then it enters the loop
```
k = 0;
for A times {
  k = a[k];
}
```

* As a specific example, let's say $N=576$ ($=$ 72 elements), $S=128$ ($=$ 16 elements), then the above loop will access

```
a[0], a[16], a[32], a[48], a[64], a[8], a[24], a[40], a[56], a[0], a[16], a[32], a[48], a[64], a[8], a[24], a[40], a[56], ... (repeats) ...
```

* since accessing an element (say a[0]) will bring the entire 64-byte block containing it into the cache and our primary interest in cache behavior, we set parameters so that we access only a single element in each 64-byte block.  
* to that end, we typically specify $S$ be a multiple of 64
* for a similar reason $N$ is also typically a multiple of 64
* if a largest common divisor of $N$ and $S$ is 64, then the loop will touch every 64-byte block of the array (as it did in the above example)
* if they have a larger common divisor (e.g., 128), there are 64-byte blocks of the array that are not accessed at all by the above loop 
* for example, when $N=512$ and $S=128$ (GCD($N$, $S$) = 128), then the loop accesses `a[0], a[16], a[32], a[48], a[0], a[16], a[32], a[48], ...(repeats)...` leaving the 64-byte block containing `a[8], a[24], a[40], a[56]` not accessed
* to help you understand what's going on, the program shows the number of distinct 64-byte blocks accessed in the line `distinct blocks`


* will touch all blocks

In [ ]:
! ./ptr_chase -n $((16 * 1024)) -s 64 -a $((100 * 1000 * 1000))

* will touch only half of the blocks (GCD(16 * 1024, 128) = 128)

In [ ]:
! ./ptr_chase -n $((16 * 1024)) -s 128 -a $((100 * 1000 * 1000))

* will touch only a forth of the blocks (GCD(16 * 1024, 256) = 256)

In [ ]:
! ./ptr_chase -n $((16 * 1024)) -s 256 -a $((100 * 1000 * 1000))

* will touch all blocks despite S=256, as GCD(16 * 1024 + 64, 256) = 64

In [ ]:
! ./ptr_chase -n $((16 * 1024 + 64)) -s 256 -a $((100 * 1000 * 1000))


* if you cyclically accesses the set of data that do not fit a level of cache (e.g., L1 cache), then you will observe the cost of data access going up
* 
* change the argument given to `-n` (16 * 1024) in the following and see what happens on metrics reported (instructions, cycles, cache misses)


In [ ]:
! ./ptr_chase -n $((16 * 1024)) -s 64 -a $((100 * 1000 * 1000))


# <font color="green"> Problem 1 :  Find out the size and latency of caches/memory</font>
* the latency per access we observed above indicates L1 cache latency (as the data size is smaller than L1 cache)
* by increasing the data size (`-n`), find out the data size around which the latency as well as the number of L1 cache misses sharply increase from the above experiment
* it indicates the L1 cache size and the latency of the L2 cache
* you may increase the data size further to find out L2 cache size and L3 cache latency, and L3 cache size and the main memory latency
* to the extend possible try to find them out
 
* you may be able to Google cache sizes of the processor we are using (Intel(R) Xeon(R) Platinum 8368 CPU @ 2.40GH) but are encouraged to _observe_ them by yourself
* also, get a sense of latency of each memory hierarchy


In [ ]:
! ./ptr_chase -n $((16 * 1024)) -s 64 -a $((100 * 1000 * 1000))

BEGIN SOLUTION
END SOLUTION

|  |size|latency|
|--|----|-------|
|L1|    |       |
|L2|    |       |
|L3|    |       |
|main memory|X|       |




# 2. Visualizing latency
* run the program with many data sizes and visualize the latency across the whole range
* here is a helper program that runs the program with many data sizes and strides


In [ ]:
#!/usr/bin/python3
import os

def is_prime(x):
    """ return if x is prime or not """
    if x == 1:
        return 0
    for d in range(2, x):
        if x % d == 0:
            return 0
        if d * d > x:
            return 1
    return 1
    
def find_prime(a):
    """ find the smallest prime >= a """
    for x in range(a, 100 * a):
        if is_prime(x):
            return x
    assert(0)

def seq(a, b, r, u, prime):
    """
    generate numbers a, ar, ar^2, ar^3, ..., b
    but make each number a multiple of u.
    also, if prime=True, make each number
    a prime multiple of u.
    """
    S = []
    x = a
    while x <= b:
        y = (int(x) + u - 1) // u
        z = find_prime(y) if prime else y
        if z * u <= b:
            S.append(z * u)
        x = x * r
    return sorted(set(S))

def run1(cmd):
    print(cmd)
    return os.system(cmd)

def run(cmd, sizes, strides, a=10*1000*1000, repeats=3):
    os.makedirs("out", exist_ok=True)
    for n in sizes:
        for s in strides:
            for t in range(repeats):
                r = run1("{cmd} -n {n} -s {s} -a {a} > out/out_{n}_{s}_{t}.txt"
                         .format(cmd=cmd, n=n, s=s, a=a, t=t))
                if r:
                    return r
    print("done")
    return 0


if 0:                           #  e.g.,
    run("./ptr_chase",
        # generate a geometric sequence from 2^10 to 2^28
        seq(2 ** 10, 2 ** 29, 1.3, 64, 1),
        # make stride always 64
        [ 64, 4160 ])
    


* `seq(a, b, r, u, 0)` generates a geometric sequence of the ratio $r$ starting near $a$ and ending near $b$
* each number is made a multiple of $u$
* `seq(a, b, r, u, 1)` is similar to `seq(a, b, r, u, 0)`, except each number is made a _prime_ multiple of $u$


In [ ]:
seq(2 ** 10, 2 ** 20, 1.3, 64, 1)


* we like to test the whole range of data sizes from sizes smaller than L1 cache (a few K bytes) to those much larger than L3 cache (several hundreds of M bytes)

* we make the stride large (4096) for a reason described later
* the purpose is to make it look like random accesses from the processor's point of view
* as the sizes are generated as a prime multiple of 64, GCD(size, stride) is always 64, ensuring all 64-byte blocks are accessed


In [ ]:
run("./ptr_chase",
    # generate a geometric sequence from 2^10 to 2^28
    seq(2 ** 10, 2 ** 28, 1.3, 64, 1),
    # make stride always 64
    [ 4096 ])


* data are dumped into many files in out/out_*.txt


In [ ]:
! ls out/


* data reader


In [ ]:
#!/usr/bin/python3
import re
import pandas as pd

def mk_key(s):
    s = s.strip()
    s = s.replace(" ", "_")
    return s

def mk_val(s):
    try:
        return int(s)
    except ValueError:
        pass
    try:
        return float(s)
    except ValueError:
        pass
    return s

def read_file(filename):
    data = {}
    pat = re.compile("(?P<key>[^:]+):(?P<val>[^:]+)")
    with open(filename) as fp:
        for line in fp:
            m = pat.match(line)
            if m:
                key = mk_key(m.group("key"))
                val = mk_val(m.group("val"))
                data[key] = val
    return data

def read_files(filenames):
    data = []
    for filename in filenames:
        data.append(read_file(filename))
    return pd.DataFrame(data)




* visualizer for size vs latency


In [ ]:
#!/usr/bin/python3
import glob
import matplotlib.pyplot as plt
import numpy as np

#from read_data import read_files

def size_vs_what(filenames, what="cycles", strides=None):
    # read all data
    dfa = read_files(filenames)
    fig, ax = plt.subplots()
    if strides is None:
        strides = dfa["stride"].unique()
    for stride in strides:
        # get data of the specified stride
        df = dfa[dfa["stride"] == stride]
        avg = df.groupby(["size"]).mean()
        avg = avg.sort_values("size")
        size = np.array(avg.index)
        vals = np.array(avg[what]) / np.array(avg["accesses"])
        label = "stride={stride}".format(stride=stride)
        line, = ax.plot(size, vals, "*-", label=label)
    ax.set_xlabel("size")
    ax.set_ylabel("{what} per access".format(what=what))
    plt.legend()
    plt.xscale("log")
    plt.ylim(0)
    plt.show()

#size_vs_what(glob.glob("out/out_*.txt"), "cycles")
#size_vs_what(glob.glob("out/out_*.txt"), "L1-dcache-load-misses")
#size_vs_what(glob.glob("out/out_*.txt"), "cache-misses")

#size_vs_what(glob.glob("out/out_*.txt"), "l2_lines_in.all")
# size_vs_what(glob.glob("out/out_*.txt"), "offcore_requests.l3_miss_demand_data_rd")



* let's visualize it


In [ ]:
size_vs_what(glob.glob("out/out_*.txt"))


* from the visualization, find again the size and latency of each level of the caches and the latency of main memory



* the same program can visualize cache misses (measured by Linux perf, which uses hardware counters to count cache misses)


In [ ]:
size_vs_what(glob.glob("out/out_*.txt"), what="L1-dcache-load-misses")
size_vs_what(glob.glob("out/out_*.txt"), what="cache-misses")


# 3. Prefetching
* run the same experiment again, but this time with a small stride (64)


In [ ]:
run("./ptr_chase",
    # generate a geometric sequence from 2^10 to 2^28
    seq(2 ** 10, 2 ** 28, 1.3, 64, 1),
    # make stride always 64
    [ 64 ])


* let's visualize it


In [ ]:
size_vs_what(glob.glob("out/out_*.txt"))


* why is the latency of the case stride=64 much smaller?
* it's not that individual accesses literally take shorter from the memory (or whichever cache level the data happens to be in) to the processor, but that the processor detected sequential access and fetch data earlier than the corresponding load instruction is actually issued
* it happens only within the same OS page (4KB), so with stride $\geq 4096$, the effect of prefetching becomes none and even for strides $< 4096$, the effect of prefetching decreases as the next data is more likely to be on a different page
* if you like, run the experiments with various strides (give a list of strides you want to use as the third parameters of run, like run(..., ..., [256, 1024, 16384])


In [ ]:
size_vs_what(glob.glob("out/out_*.txt"), what="L1-dcache-load-misses")
size_vs_what(glob.glob("out/out_*.txt"), what="cache-misses")


# 4. Translating it to bandwidth
* visualize the same result with a bandwidth perspective
* it is simply the number of data accessed in total divided by the number of cycles
  * the former is 64 $\times$ the number of iterations (given as `-a`).  64 is the cache line size. we count a single element access as 64 bytes even if we actually touch only 8 bytes of it, because the processor brings the entire cache line anyways


In [ ]:
#!/usr/bin/python3
import glob
import matplotlib.pyplot as plt
import numpy as np

#from read_data import read_files

def size_vs_bw(filenames, strides=None):
    # read all data
    dfa = read_files(filenames)
    fig, ax = plt.subplots()
    if strides is None:
        strides = dfa["stride"].unique()
    for stride in strides:
        # get data of the specified stride
        df = dfa[dfa["stride"] == stride]
        avg = df.groupby(["size"]).mean()
        avg = avg.sort_values("size")
        size = np.array(avg.index)
        vals = np.array(avg["accesses"]) * 64 / np.array(avg["cycles"])
        label = "stride={stride}".format(stride=stride)
        line, = ax.plot(size, vals, "*-", label=label)
    ax.set_xlabel("size")
    ax.set_ylabel("bytes/cycle")
    plt.legend()
    plt.xscale("log")
    plt.ylim(0)
    plt.show()

#size_vs_what(glob.glob("out/out_*.txt"), "cycles")
#size_vs_what(glob.glob("out/out_*.txt"), "L1-dcache-load-misses")
#size_vs_what(glob.glob("out/out_*.txt"), "cache-misses")

#size_vs_what(glob.glob("out/out_*.txt"), "l2_lines_in.all")
# size_vs_what(glob.glob("out/out_*.txt"), "offcore_requests.l3_miss_demand_data_rd")


In [ ]:
size_vs_bw(glob.glob("out/out_*.txt"))
size_vs_bw(glob.glob("out/out_*.txt"))


* recall how much was enough to get close to processor's peak performance for SpMV, if we (conservatively) assume a single fma takes accessing 8 bytes (a single DP element, only counting the matrix element)
* as the processor's peak is 16 DP fmas/cycle, it needs to bring 8 x 16 = 128 bytes/cycle to the processor
* it is far from the numbers we are seeing, even if data are coming from L1 cache
* if you multiply the processor's frequency (2.4GHz) to the bytes/cycle, you get the bandwidth in terms of bytes/sec
  * equivalently you multiply bytes/cycle by 2.4 to get GB/sec
* observe that the main memory bandwidth, even with the small stride (64) where prefetching is going on, it is just a few GB/sec, far from what the processor's spec sheet advertises



# 5. Why you'd better avoid large-power-of-two strides?
* a data element is deterministically mapped to a particular _set_ in the cache and it is the data in each set that are managed by LRU
* if data you are actively using are mapped on the same set, then the cache miss sharply increase, even if the total size of those data is much smaller than the cache (conflict misses)
* in the case of 12-way set associative, 48KB L1 cache, data whose addresses are apart by a multiple of 4KB (48KB/12) are mapped on the same cache set, therefore if you cyclically access (12 + 1) elements with 4KB stride, it induces almost 100% cache misses, even if the total data size actually used is just 64 x 13 = 832 bytes
  
* contrast the following two
* observe the "distinct_blocks" reported

In [ ]:
! ./ptr_chase -n $((4096 * 11)) -s 4096 -a $((100 * 1000 * 1000))
! ./ptr_chase -n $((4096 * 13)) -s 4096 -a $((100 * 1000 * 1000))


* stride accesses happen most typically for multidimensional arrays
* if, for example, you have 2D array (matrix) of DP numbers that has 1024 elements in a row, two adjacent elements in a column are 8192 bytes apart in the row-major order
* therefore scanning elements in the column direction (i.e., `a(i,j), a(i+1,j), a(i+2,j), ...`) would access them in a large-power-of-two stride
* this access pattern appears in matrix-matrix multiplication for matrix B (of A * B)
* to avoid this effect, it's good idea to adjust the number of elements in a row by padding (make the matrix a bit fatter than what it actually is, to make the address distance between two adjacent elements in a column not a large power of two). 64 $\times$ an odd number is a safe choice